In [1]:
# ==============================================
# RENT VS BUY — FULLY REFACTORED, TESTABLE ENGINE
# ==============================================
# Key properties:
# - No global mutation
# - Annual → monthly return conversion done correctly
# - Mortgage + HELOC tracked separately
# - Proper re-amortization when balances change
# - Inflation-adjusted assessments
# - TFSA / RRSP room rules centralized
# - Deterministic core engine (randomness injected)
# - Each component unit-testable in isolation

from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
import copy

# =========================
# CONFIGURATION STRUCTURES
# =========================

@dataclass
class RentParams:
    start_rent: float
    growth: float

@dataclass(frozen=True)
class MarketParams:
    annual_return: float
    annual_volatility: float
    inflation: float

    def monthly_return(self, annual_r):
        return (1 + annual_r) ** (1/12) - 1


@dataclass(frozen=True)
class IncomeParams:
    starting_salary: float
    salary_growth: float
    avg_tax_rate: float


@dataclass(frozen=True)
class LivingCostParams:
    base_monthly: float
    growth: float


@dataclass(frozen=True)
class PropertyParams:
    price: float
    down_pct: float
    mortgage_rate: float
    amort_years: int
    appreciation: float
    property_tax_rate: float
    condo_fee: float
    maintenance_pct: float
    selling_cost_pct: float


@dataclass(frozen=True)
class RegisteredAccountRules:
    tfsa_start_room: float
    tfsa_annual_room: float
    rrsp_start_room: float
    rrsp_max_annual: float
    rrsp_pct_income: float


@dataclass(frozen=True)
class AssessmentRisk:
    prob_start: float
    prob_increment: float
    min_cost: float
    max_cost: float


# =========================
# FINANCIAL PRIMITIVES
# =========================

class Mortgage:
    def __init__(self, principal, rate, amort_months):
        self.rate = rate / 12
        self.balance = principal
        self.remaining_months = amort_months
        self.payment = self._calc_payment()

    def _calc_payment(self):
        if self.balance <= 0:
            return 0
        r = self.rate
        n = self.remaining_months
        return (self.balance * r * (1 + r) ** n) / ((1 + r) ** n - 1)

    def step(self):
        if self.balance <= 0:
            return 0, 0
        interest = self.balance * self.rate
        principal = min(self.payment - interest, self.balance)
        self.balance -= principal
        self.remaining_months -= 1
        return interest, principal

    def refinance(self, amount):
        self.balance += amount
        self.remaining_months = max(self.remaining_months, 1)
        self.payment = self._calc_payment()


class HELOC:
    def __init__(self, rate):
        self.rate = rate / 12
        self.balance = 0

    def step(self):
        self.balance *= (1 + self.rate)

    def borrow(self, amount):
        self.balance += amount

    def repay(self, amount):
        pay = min(amount, self.balance)
        self.balance -= pay
        return pay


class Portfolio:
    def __init__(self, tfsa_room, rrsp_room):
        self.tfsa = 0
        self.rrsp = 0
        self.unreg = 0
        self.tfsa_room = tfsa_room
        self.rrsp_room = rrsp_room

    def grow(self, monthly_r):
        self.tfsa *= (1 + monthly_r)
        self.rrsp *= (1 + monthly_r)
        self.unreg *= (1 + monthly_r * 0.75)

    def invest(self, amount):
        if amount <= 0:
            return
        tfsa_take = min(amount, self.tfsa_room)
        self.tfsa += tfsa_take
        self.tfsa_room -= tfsa_take
        amount -= tfsa_take

        rrsp_take = min(amount, self.rrsp_room)
        self.rrsp += rrsp_take
        self.rrsp_room -= rrsp_take
        amount -= rrsp_take

        self.unreg += amount

    def withdraw_for_deficit(self, deficit, tax_rate):
        # Order: unreg → tfsa → rrsp
        take = min(deficit, self.unreg)
        self.unreg -= take
        deficit -= take

        take = min(deficit, self.tfsa)
        self.tfsa -= take
        deficit -= take

        if deficit > 0 and self.rrsp > 0:
            gross = deficit / (1 - tax_rate)
            take = min(gross, self.rrsp)
            self.rrsp -= take
            deficit -= take * (1 - tax_rate)

        return deficit

    def total(self):
        return self.tfsa + self.rrsp + self.unreg


# =========================
# ASSESSMENT GENERATOR
# =========================

def generate_assessments(risk: AssessmentRisk, years, inflation, rng):
    timeline = np.zeros(years)
    prob = risk.prob_start

    for y in range(5, years):
        if rng.random() < prob:
            raw = rng.uniform(risk.min_cost, risk.max_cost)
            timeline[y] = raw * ((1 + inflation) ** y)
            prob = risk.prob_start
        else:
            prob += risk.prob_increment

    return timeline


# =========================
# CORE SIMULATION ENGINE
# =========================

class SimulationEngine:
    def __init__(self, years, market: MarketParams, income: IncomeParams,
                 living: LivingCostParams, property_p: PropertyParams,
                 accounts: RegisteredAccountRules, assessment_risk: AssessmentRisk,
                 rent: RentParams | None = None):
        self.years = years
        self.months = years * 12
        self.market = market
        self.income = income
        self.living = living
        self.property = property_p
        self.accounts = accounts
        self.assessment_risk = assessment_risk
        self.rent = rent


    def run_buyer(self, market_returns, prop_returns, assessment_timeline):
        salary = self.income.starting_salary / 12
        living_cost = self.living.base_monthly

        portfolio = Portfolio(
            self.accounts.tfsa_start_room,
            self.accounts.rrsp_start_room
        )

        mortgage = Mortgage(
            self.property.price * (1 - self.property.down_pct),
            self.property.mortgage_rate,
            self.property.amort_years * 12
        )

        heloc = HELOC(rate=0.07)
        home_value = self.property.price

        records = []
        buyer_principal_flow = []
        bankrupt_month = None

        for m in range(self.months):
            y = m // 12

            # Market growth
            portfolio.grow(self.market.monthly_return(market_returns[y]))

            # Mortgage + HELOC
            interest, principal = mortgage.step()
            heloc.step()

            # Costs
            property_tax = home_value * self.property.property_tax_rate / 12
            maintenance = (
                self.property.price * self.property.maintenance_pct / 12
            ) * ((1 + self.market.inflation) ** y)
            condo = self.property.condo_fee * ((1 + self.market.inflation) ** y)
            assessment = assessment_timeline[y] if m % 12 == 0 else 0

            housing_cost = (
                mortgage.payment + property_tax + maintenance + condo + assessment
            )

            buyer_principal_flow.append(housing_cost)
            
            net_income = salary * (1 - self.income.avg_tax_rate)
            surplus = net_income - living_cost - housing_cost

            if surplus < 0:
                deficit = portfolio.withdraw_for_deficit(
                    -surplus, self.income.avg_tax_rate
                )
                if deficit > 0:
                    max_ltv = home_value * 0.8
                    avail = max_ltv - (mortgage.balance + heloc.balance)
                    draw = min(avail, deficit)
                    heloc.borrow(draw)
                    deficit -= draw
            else:
                # 1. Try to pay 100% to HELOC, but capture what was ACTUALLY paid
                target_repayment = surplus
                actual_repayment = heloc.repay(target_repayment) 
                
                # 2. Invest the rest (Original Surplus - What we actually used)
                # If actual_repayment was 0, this invests 100% of the surplus.
                portfolio.invest(surplus - actual_repayment)

            # Bankruptcy condition
            if portfolio.total() < 0:
                bankrupt_month = m
                break

            equity = (
                home_value
                - mortgage.balance
                - heloc.balance
                - (home_value * self.property.selling_cost_pct)
            )

            records.append(portfolio.total() + equity)

            # Annual updates
            if m % 12 == 11:
                salary *= (1 + self.income.salary_growth)
                living_cost *= (1 + self.living.growth)
                home_value *= (1 + prop_returns[y])

                portfolio.tfsa_room += self.accounts.tfsa_annual_room
                portfolio.rrsp_room += min(
                    self.accounts.rrsp_pct_income * salary * 12,
                    self.accounts.rrsp_max_annual
                )

        return (
            pd.Series(records),
            pd.Series(buyer_principal_flow),
            bankrupt_month
        )



# =========================
# RENTER STRATEGY ENGINE
# =========================

class RenterStrategy:
    FIXED_PCT = "FIXED_PCT"
    MAX_AVAILABLE = "MAX_AVAILABLE"
    MATCH_BUYER = "MATCH_BUYER"


class RenterEngine:
    def __init__(self, strategy, buyer_reference=None, fixed_pct=0.30):
        self.strategy = strategy
        self.fixed_pct = fixed_pct
        self.buyer_reference = buyer_reference

    def target_investment(self, surplus, salary_monthly, month):
        if surplus <= 0:
            return 0
        if self.strategy == RenterStrategy.MAX_AVAILABLE:
            return surplus
        if self.strategy == RenterStrategy.FIXED_PCT:
            return min(salary_monthly * self.fixed_pct, surplus)
        if self.strategy == RenterStrategy.MATCH_BUYER:
            if self.buyer_reference is None:
                raise ValueError("MATCH_BUYER requires buyer reference series")
            return min(self.buyer_reference.iloc[month], surplus)
        raise ValueError("Unknown renter strategy")


# =========================
# BANKRUPTCY MODEL
# =========================

class BankruptcyException(Exception):
    pass


@dataclass
class BankruptcyPolicy:
    min_liquidity: float
    max_debt_ratio: float  # debt / income

    def check(self, portfolio_value, total_debt, monthly_income):
        if portfolio_value < self.min_liquidity:
            raise BankruptcyException("Liquidity exhausted")
        if monthly_income > 0 and (total_debt / (monthly_income * 12)) > self.max_debt_ratio:
            raise BankruptcyException("Debt spiral")


# =========================
# EXTENDED SIMULATION ENGINE
# =========================

class ExtendedSimulationEngine(SimulationEngine):
    def run_renter(self, market_returns, assessment_timeline, 
                 renter_engine: RenterEngine, 
                 bankruptcy_policy: BankruptcyPolicy,
                 initial_capital: float): # <--- ADDED ARGUMENT

        if self.rent is None:
            raise ValueError("RentParams must be provided to engine")

        salary = self.income.starting_salary / 12
        living_cost = self.living.base_monthly
        rent = self.rent.start_rent

        # Initialize Portfolio with the cash the buyer spent on the house
        portfolio = Portfolio(self.accounts.tfsa_start_room, 
                              self.accounts.rrsp_start_room)
        portfolio.invest(initial_capital) # <--- CRITICAL FIX

        records = []
        bankrupt_month = None

        for m in range(self.months):
            y = m // 12
            monthly_r = self.market.monthly_return(market_returns[y])
            portfolio.grow(monthly_r)

            assessment = assessment_timeline[y] if m % 12 == 0 else 0
            housing_cost = rent + assessment

            net_income = salary * (1 - self.income.avg_tax_rate)
            surplus = net_income - living_cost - housing_cost

            if surplus < 0:
                # Handle deficits (same as before)
                deficit = portfolio.withdraw_for_deficit(-surplus, self.income.avg_tax_rate)
                if deficit > 0 and bankrupt_month is None:
                    bankrupt_month = m
                    break
            else:
                # Investment logic
                invest_amt = renter_engine.target_investment(surplus, salary, m)
                portfolio.invest(invest_amt)
                # Note: Any remaining surplus here is "lifestyle creep" (spent)

            # Bankruptcy Check
            total_debt = 0
            try:
                bankruptcy_policy.check(portfolio.total(), total_debt, salary)
            except BankruptcyException:
                bankrupt_month = m
                break

            records.append(portfolio.total())

            # Annual updates (Inflation, Raises, etc.)
            if m % 12 == 11:
                salary *= (1 + self.income.salary_growth)
                living_cost *= (1 + self.living.growth)
                rent *= (1 + self.rent.growth)
                portfolio.tfsa_room += self.accounts.tfsa_annual_room
                portfolio.rrsp_room += min(self.accounts.rrsp_pct_income * salary * 12, 
                                           self.accounts.rrsp_max_annual)

        # Pad results if bankrupt
        if bankrupt_month is not None:
            return pd.Series(records + [0] * (self.months - len(records))), bankrupt_month
        return pd.Series(records), None


# =========================
# COHORT-LEVEL MONTE CARLO
# =========================

def run_cohort(engine: ExtendedSimulationEngine, runs, renter_strategies,
               bankruptcy_policy, rng_seed=42):

    rng = np.random.default_rng(rng_seed)

    results = {"BUYER": [], **{s: [] for s in renter_strategies}}
    bankruptcies = {"BUYER": [], **{s: [] for s in renter_strategies}}

    # CALCULATE INITIAL CAPITAL (The "Opportunity Cost")
    initial_capital = (engine.property.price * engine.property.down_pct) + \
                      (engine.property.price * engine.property.selling_cost_pct) 
                      # Note: using selling_cost as proxy for closing cost here

    for i in range(runs):
        market_returns = rng.normal(
            engine.market.annual_return,
            engine.market.annual_volatility,
            engine.years
        )

        prop_returns = rng.normal(
            engine.property.appreciation,
            0.05,
            engine.years
        )

        assessments = generate_assessments(
            engine.assessment_risk,
            engine.years,
            engine.market.inflation,
            rng
        )

        # ---------- BUYER ----------
        buyer_series, buyer_principal, buyer_bk = engine.run_buyer(
            market_returns,
            prop_returns,
            assessments
        )

        results["BUYER"].append(buyer_series.iloc[-1])
        bankruptcies["BUYER"].append(buyer_bk)

        # ---------- RENTERS ----------
        for strat in renter_strategies:
            renter_engine = RenterEngine(
                strategy=strat,
                buyer_reference=buyer_principal
            )

            series, bk = engine.run_renter(
                market_returns=market_returns,
                assessment_timeline=assessments,
                renter_engine=renter_engine,
                bankruptcy_policy=bankruptcy_policy,
                initial_capital=initial_capital
            )

            # If bankrupt, we record 0 net worth for easier stats, or the last value
            final_val = series.iloc[-1] if bk is None else 0
            results[strat].append(final_val)
            bankruptcies[strat].append(bk)

    return results, bankruptcies

def summarize_strategy(name, values, bankruptcies):
    values = np.array(values)
    bankrupt = np.array([b is not None for b in bankruptcies])

    return {
        "Strategy": name,
        "Mean Net Worth": np.mean(values),
        "Median Net Worth": np.median(values),
        "10th %ile": np.percentile(values, 10),
        "90th %ile": np.percentile(values, 90),
        "Worst Case": np.min(values),
        "Best Case": np.max(values),
        "Bankruptcy Rate": bankrupt.mean(),
        "Survival Rate": 1 - bankrupt.mean()
    }

def win_rate(strategy):
    renter = np.array(results[strategy])
    return np.mean(renter > buyer)

def survivors_only(values, bankruptcies):
    return np.array([
        v for v, b in zip(values, bankruptcies) if b is None
    ])

def survival_adjusted_value(values, bankruptcies):
    survive = survivors_only(values, bankruptcies)
    return np.mean(survive) if len(survive) > 0 else np.nan



In [2]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots


engine = ExtendedSimulationEngine(
    rent=RentParams(
        start_rent=2600,
        growth=0.03
    ),
    years=30,
    market=MarketParams(0.06, 0.08, 0.02),
    income=IncomeParams(150000, 0.03, 0.30),
    living=LivingCostParams(1000, 0.02),
    property_p=PropertyParams(
        price=650000,
        down_pct=0.10,
        mortgage_rate=0.045,
        amort_years=30,
        appreciation=0.03,
        property_tax_rate=0.0061,
        condo_fee=650,
        maintenance_pct=0.005,
        selling_cost_pct=0.05
    ),
    accounts=RegisteredAccountRules(
        tfsa_start_room=80000,
        tfsa_annual_room=7000,
        rrsp_start_room=50000,
        rrsp_max_annual=32000,
        rrsp_pct_income=0.18
    ),
    assessment_risk=AssessmentRisk(0.02, 0.03, 5000, 35000)
)

bankruptcy_policy = BankruptcyPolicy(
    min_liquidity=5000,
    max_debt_ratio=6
)

results, bankruptcies = run_cohort(
    engine,
    runs=500,
    renter_strategies=[
        RenterStrategy.MAX_AVAILABLE,
        RenterStrategy.FIXED_PCT,
        RenterStrategy.MATCH_BUYER
    ],
    bankruptcy_policy=bankruptcy_policy
)


# ==========================================
# 1. VERBOSE PARAMETER PRINTING
# ==========================================
def print_parameters(engine, policy):
    p = engine.property
    m = engine.market
    r = engine.rent
    i = engine.income
    ar = engine.assessment_risk
    
    print("="*65)
    print(f"SIMULATION CONFIGURATION (30 Years / {engine.months} Months)")
    print("="*65)
    print(f"[PROPERTY]  Price: ${p.price:,.0f} | Down: {p.down_pct:.0%} | Rate: {p.mortgage_rate:.2%}")
    print(f"            Tax: {p.property_tax_rate:.2%} | Maint: {p.maintenance_pct:.1%} | Condo Fee: ${p.condo_fee}/mo")
    print(f"[RENT]      Start: ${r.start_rent}/mo | Growth: {r.growth:.1%}")
    print(f"[MARKET]    Return: {m.annual_return:.1%} | Volatility: {m.annual_volatility:.1%} | Inflation: {m.inflation:.1%}")
    print(f"[RISK]      Assessments: {ar.prob_start:.1%} prob, ${ar.min_cost/1000:.0f}k-${ar.max_cost/1000:.0f}k range")
    print(f"[POLICY]    Bankruptcy: <${policy.min_liquidity} Liq or >{policy.max_debt_ratio}x Debt/Income")
    print("="*65 + "\n")


# ==========================================
# 2. a) BASE CASE (DETERMINISTIC) RUNNER
# ==========================================
def run_deterministic_base_case(engine, strategies, policy):
    """
    Runs the simulation ONCE with zero volatility (perfect averages).
    Useful to see the 'theoretical' geometric drift.
    """
    # 1. Create constant (flat) arrays for returns
    # We use the MEAN values for everything.
    flat_market_ret = np.full(engine.years, engine.market.annual_return)
    flat_prop_ret = np.full(engine.years, engine.property.appreciation)
    
    # 2. Smooth out assessments (Expected Value distributed evenly)
    # Approx expected cost = probability * (min + max) / 2
    avg_assess_cost = engine.assessment_risk.prob_start * (engine.assessment_risk.min_cost + engine.assessment_risk.max_cost) / 2
    # We'll just apply this small cost annually to be fair
    flat_assessments = np.zeros(engine.years)
    flat_assessments[5:] = avg_assess_cost # Start after year 5 like the risk model
    
    # 3. Initial Capital (Opportunity Cost)
    initial_capital = (
        engine.property.price * engine.property.down_pct + 
        engine.property.price * engine.property.selling_cost_pct
    )

    results = {}
    
    # BUYER
    b_series, b_flow, _ = engine.run_buyer(flat_market_ret, flat_prop_ret, flat_assessments)
    b_flow_series = pd.Series(b_flow)
    results["BUYER"] = b_series.values

    # RENTERS
    for strat in strategies:
        r_eng = RenterEngine(strat, b_flow_series)
        r_series, _ = engine.run_renter(
            market_returns=flat_market_ret,
            assessment_timeline=flat_assessments,
            renter_engine=r_eng,
            bankruptcy_policy=policy,
            initial_capital=initial_capital
        )
        results[strat] = r_series.values
        
    return results

# ==========================================
# 2. EXECUTION (Running the Cohort)
# ==========================================

def run_cohort_extended(engine, runs, strategies, policy):
    rng = np.random.default_rng(42)
    history = {"BUYER": [], **{s: [] for s in strategies}}
    
    # Track Assessment Statistics
    assessment_stats = {
        "count": [],
        "total_cost": []
    }

    # Calculate Initial Capital for Renters (Opportunity Cost)
    initial_capital = (
        engine.property.price * engine.property.down_pct + 
        engine.property.price * engine.property.selling_cost_pct
    )
    
    print(f"Starting {runs} Simulation Runs...")
    print(f"Renter Initial Investment (Opp. Cost): ${initial_capital:,.2f}")

    for _ in range(runs):
        m_ret = rng.normal(engine.market.annual_return, engine.market.annual_volatility, engine.years)
        p_ret = rng.normal(engine.property.appreciation, 0.05, engine.years)
        assess = generate_assessments(engine.assessment_risk, engine.years, engine.market.inflation, rng)
        
        # Track assessments for this run
        assessment_stats["count"].append(np.count_nonzero(assess))
        assessment_stats["total_cost"].append(np.sum(assess))

        # BUYER
        b_series, b_flow, b_bk = engine.run_buyer(m_ret, p_ret, assess)
        
        # FIX: Convert b_flow list to Series so .iloc works in RenterEngine
        b_flow_series = pd.Series(b_flow)

        if len(b_series) < engine.months: # Pad if bankrupt
            pad = pd.Series([0] * (engine.months - len(b_series)))
            b_series = pd.concat([b_series, pad], ignore_index=True)
        history["BUYER"].append(b_series.values)

        # RENTERS
        for strat in strategies:
            r_eng = RenterEngine(strat, b_flow_series) # Pass the Series, not the list
            
            # Injecting manual initial_capital fix
            r_series, r_bk = engine.run_renter(
                market_returns=m_ret, 
                assessment_timeline=assess, 
                renter_engine=r_eng, 
                bankruptcy_policy=policy,
                initial_capital=initial_capital 
            )
            history[strat].append(r_series.values)
            
    return history, assessment_stats

# --- EXECUTE ---

print_parameters(engine, bankruptcy_policy)
strategies = [RenterStrategy.MAX_AVAILABLE, RenterStrategy.MATCH_BUYER, RenterStrategy.FIXED_PCT]

print(f"1. Running Deterministic Base Case (No Volatility)...")
base_case_results = run_deterministic_base_case(engine, strategies, bankruptcy_policy)

print(f"2. Running Monte Carlo Cohort (500 Runs with Volatility)...")
data, assess_data = run_cohort_extended(engine, 500, strategies, bankruptcy_policy)

# ==========================================
# 3. STATISTICS & VISUALIZATION
# ==========================================

# --- A. Strategy Stats Table ---
stats_rows = []
for strat in ["BUYER"] + strategies:
    runs = data[strat]
    finals = [r[-1] for r in runs]
    survivors = [x for x in finals if x > 0]
    
    win_rate = "N/A"
    if strat != "BUYER":
        buyer_finals = [r[-1] for r in data["BUYER"]]
        wins = sum(r > b for r, b in zip(finals, buyer_finals))
        win_rate = f"{(wins/len(runs))*100:.1f}%"

    stats_rows.append({
        "Strategy": strat,
        "Win Rate (vs Buyer)": win_rate,
        "Median Net Worth": f"${np.median(survivors):,.0f}",
        "Mean Net Worth": f"${np.mean(survivors):,.0f}",
        "Survival Rate": f"{(len(survivors)/len(runs))*100:.1f}%"
    })

df_stats = pd.DataFrame(stats_rows)

# --- B. Assessment Stats ---
mean_assess_count = np.mean(assess_data["count"])
mean_assess_cost = np.mean(assess_data["total_cost"])
max_assess_cost = np.max(assess_data["total_cost"])

print(f"\n[ASSESSMENT STATISTICS]")
print(f"Avg Frequency: {mean_assess_count:.1f} assessments over 30 years")
print(f"Avg Total Cost: ${mean_assess_cost:,.2f}")
print(f"Worst Case Cost: ${max_assess_cost:,.2f}")

# --- C. PLOTLY VISUALIZATIONS ---

colors = {
    "BUYER": "rgb(31, 119, 180)",          # Blue
    "MAX_AVAILABLE": "rgb(255, 127, 14)",  # Orange
    "MATCH_BUYER": "rgb(44, 160, 44)",     # Green
    "FIXED_PCT": "rgb(214, 39, 40)"        # Red
}

x_axis = np.arange(engine.months) / 12

# --- CHART A: BASE CASE (DETERMINISTIC) ---
fig_base = go.Figure()
for strat in ["BUYER"] + strategies:
    fig_base.add_trace(go.Scatter(
        x=x_axis, y=base_case_results[strat],
        mode='lines', name=strat,
        line=dict(color=colors[strat], width=3)
    ))

fig_base.update_layout(
    title="<b>Base Case Trajectory</b> (Zero Volatility / Perfect Averages)",
    xaxis_title="Years",
    yaxis_title="Net Worth ($)",
    template="plotly_white",
    height=500,
    hovermode="x unified"
)
fig_base.show()

# 1. COMBINED TRAJECTORY CHART
fig_main = go.Figure()

for strat in ["BUYER"] + strategies:
    matrix = np.array(data[strat])
    p50 = np.median(matrix, axis=0) # Median Line
    
    fig_main.add_trace(go.Scatter(
        x=x_axis, y=p50, 
        mode='lines', 
        name=strat, 
        line=dict(color=colors[strat], width=3)
    ))

fig_main.update_layout(
    title="<b>Median Net Worth Trajectory</b> (Comparison)",
    xaxis_title="Years",
    yaxis_title="Net Worth ($)",
    template="plotly_white",
    height=500,
    hovermode="x unified",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)
fig_main.show()

# 2. ASSESSMENT COST DISTRIBUTION
fig_assess = go.Figure()
fig_assess.add_trace(go.Histogram(
    x=assess_data["total_cost"],
    nbinsx=40,
    marker_color='purple',
    opacity=0.7,
    name="Total Cost"
))
fig_assess.update_layout(
    title=f"<b>Special Assessment Risk Profile</b><br>Distribution of 30-Year Total Costs (Avg: ${mean_assess_cost:,.0f})",
    xaxis_title="Total Assessment Cost ($)",
    yaxis_title="Frequency (Simulations)",
    template="plotly_white",
    height=400
)
fig_assess.show()

# 3. SURVIVOR BOX PLOT
fig_box = go.Figure()
for strat in ["BUYER"] + strategies:
    # Filter only survivors for fair comparison in box plot
    vals = [r[-1] for r in data[strat] if r[-1] > 0]
    fig_box.add_trace(go.Box(y=vals, name=strat, marker_color=colors[strat], boxpoints=False))

fig_box.update_layout(
    title="<b>Final Net Worth Distribution</b> (Survivors Only)",
    yaxis_title="Net Worth ($)",
    template="plotly_white",
    height=450
)
fig_box.show()

# 4. STATS TABLE
fig_tbl = go.Figure(data=[go.Table(
    header=dict(values=list(df_stats.columns), fill_color='paleturquoise', align='left'),
    cells=dict(values=[df_stats[k].tolist() for k in df_stats.columns], fill_color='lavender', align='left')
)])
fig_tbl.update_layout(title="<b>Complete Statistical Summary</b>", height=300)
fig_tbl.show()

# --- CHART 6: NET WORTH DENSITY (NEW!) ---
fig_density = go.Figure()
for strat in ["BUYER"] + strategies:
    # Filter survivors only for clean distribution
    vals = [r[-1] for r in data[strat] if r[-1] > 0]
    
    fig_density.add_trace(go.Histogram(
        x=vals,
        name=strat,
        marker_color=colors[strat],
        opacity=0.6,
        histnorm='probability density', # Normalizes height so area = 1
        xbins=dict( # Auto-binning optimization
            start=min(vals),
            end=max(vals),
            size=(max(vals) - min(vals)) / 50
        )
    ))

fig_density.update_layout(
    title="<b>6. Net Worth Density (The 'Luck' Curve)</b>",
    xaxis_title="Final Net Worth ($)",
    yaxis_title="Probability Density",
    barmode='overlay', # Shows overlaps clearly
    template="plotly_white",
    height=500,
    legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99)
)
fig_density.show()

SIMULATION CONFIGURATION (30 Years / 360 Months)
[PROPERTY]  Price: $650,000 | Down: 10% | Rate: 4.50%
            Tax: 0.61% | Maint: 0.5% | Condo Fee: $650/mo
[RENT]      Start: $2600/mo | Growth: 3.0%
[MARKET]    Return: 6.0% | Volatility: 8.0% | Inflation: 2.0%
[RISK]      Assessments: 2.0% prob, $5k-$35k range
[POLICY]    Bankruptcy: <$5000 Liq or >6x Debt/Income

1. Running Deterministic Base Case (No Volatility)...
2. Running Monte Carlo Cohort (500 Runs with Volatility)...
Starting 500 Simulation Runs...
Renter Initial Investment (Opp. Cost): $97,500.00

[ASSESSMENT STATISTICS]
Avg Frequency: 3.1 assessments over 30 years
Avg Total Cost: $89,479.30
Worst Case Cost: $209,442.90


In [3]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# ==========================================
# 1. UPDATE PORTFOLIO TO TRACK CONTRIBUTIONS
# ==========================================
# We need to monkey-patch or redefine the class to track contributions
# without breaking the existing structure.

class PortfolioWithTracking(Portfolio):
    def __init__(self, tfsa_room, rrsp_room):
        super().__init__(tfsa_room, rrsp_room)
        self.total_contributed = 0 # Track cash added

    def invest(self, amount):
        if amount <= 0: return
        self.total_contributed += amount
        super().invest(amount)

# ==========================================
# 2. UPDATED SIMULATION LOOPS (To Return Contributions)
# ==========================================

def run_cohort_with_breakdown(engine, runs, strategies, policy):
    rng = np.random.default_rng(42)
    
    # Storage for breakdown stats
    breakdown = {
        "Strategy": [],
        "Total Contributed": [],
        "Total Growth": [],
        "Final Net Worth": []
    }

    initial_capital = (
        engine.property.price * engine.property.down_pct + 
        engine.property.price * engine.property.selling_cost_pct
    )

    for _ in range(runs):
        m_ret = rng.normal(engine.market.annual_return, engine.market.annual_volatility, engine.years)
        p_ret = rng.normal(engine.property.appreciation, 0.05, engine.years)
        assess = generate_assessments(engine.assessment_risk, engine.years, engine.market.inflation, rng)
        
        # --- BUYER RUN ---
        # Re-instantiate locally to use the Tracking Portfolio
        salary = engine.income.starting_salary / 12
        living = engine.living.base_monthly
        portfolio = PortfolioWithTracking(engine.accounts.tfsa_start_room, engine.accounts.rrsp_start_room)
        mortgage = Mortgage(engine.property.price * (1 - engine.property.down_pct), engine.property.mortgage_rate, engine.property.amort_years * 12)
        heloc = HELOC(0.07)
        home_val = engine.property.price
        
        # Track Buyer Principal Payments (For Match Buyer)
        buyer_principal_flow = []
        buyer_total_principal_paid = 0

        for m in range(engine.months):
            y = m // 12
            portfolio.grow(engine.market.monthly_return(m_ret[y]))
            
            interest, principal = mortgage.step()
            buyer_principal_flow.append(principal) # Just principal for MATCH strategy logic
            buyer_total_principal_paid += principal
            
            # Costs
            costs = (home_val * engine.property.property_tax_rate / 12) + \
                    (engine.property.price * engine.property.maintenance_pct / 12 * ((1+engine.market.inflation)**y)) + \
                    (engine.property.condo_fee * ((1+engine.market.inflation)**y)) + \
                    (assess[y] if m%12==0 else 0)
            
            housing_cost = mortgage.payment + costs
            net_inc = salary * (1 - engine.income.avg_tax_rate)
            surplus = net_inc - living - housing_cost
            
            if surplus > 0:
                portfolio.invest(surplus * 0.5) # Buyer invests 50% surplus
            
            # Annual updates
            if m % 12 == 11:
                salary *= (1 + engine.income.salary_growth)
                living *= (1 + engine.living.growth)
                home_val *= (1 + p_ret[y])
        
        # Buyer Final Stats
        equity = home_val - mortgage.balance
        final_nw = portfolio.total() + equity
        # Buyer Contribution = Down Payment + Principal Paid + Portfolio Contributions
        total_contrib = initial_capital + buyer_total_principal_paid + portfolio.total_contributed
        
        breakdown["Strategy"].append("BUYER")
        breakdown["Total Contributed"].append(total_contrib)
        breakdown["Total Growth"].append(final_nw - total_contrib)
        breakdown["Final Net Worth"].append(final_nw)

        # --- RENTER RUNS ---
        b_flow_series = pd.Series(buyer_principal_flow) # Principal only match
        
        for strat in strategies:
            # Setup Renter
            r_salary = engine.income.starting_salary / 12
            r_living = engine.living.base_monthly
            r_rent = engine.rent.start_rent
            r_port = PortfolioWithTracking(engine.accounts.tfsa_start_room, engine.accounts.rrsp_start_room)
            r_port.invest(initial_capital)
            
            r_eng = RenterEngine(strat, pd.Series(b_flow_series)) # Just pass principal flow

            for m in range(engine.months):
                y = m // 12
                r_port.grow(engine.market.monthly_return(m_ret[y]))
                
                cost = r_rent + (assess[y] if m%12==0 else 0)
                net_inc = r_salary * (1 - engine.income.avg_tax_rate)
                surplus = net_inc - r_living - cost
                
                if surplus > 0:
                    # Strategy Logic
                    invest = 0
                    if strat == "MAX_AVAILABLE": invest = surplus
                    elif strat == "FIXED_PCT": invest = min(r_salary * 0.30, surplus)
                    elif strat == "MATCH_BUYER": 
                        # Match the Buyer's Principal Payment (if we have surplus)
                        ref = b_flow_series.iloc[m] if m < len(b_flow_series) else 0
                        invest = min(ref, surplus)
                    
                    r_port.invest(invest)

                # Annual updates
                if m % 12 == 11:
                    r_salary *= (1 + engine.income.salary_growth)
                    r_living *= (1 + engine.living.growth)
                    r_rent *= (1 + engine.rent.growth)
            
            breakdown["Strategy"].append(strat)
            breakdown["Total Contributed"].append(r_port.total_contributed)
            breakdown["Total Growth"].append(r_port.total() - r_port.total_contributed)
            breakdown["Final Net Worth"].append(r_port.total())

    return pd.DataFrame(breakdown)

# ==========================================
# 3. RUN & VISUALIZE
# ==========================================
print("Running Portfolio Breakdown Analysis (100 runs)...")
df_breakdown = run_cohort_with_breakdown(engine, 100, strategies, bankruptcy_policy)

# Aggregating Mean Values
df_agg = df_breakdown.groupby("Strategy").mean().reset_index()

# Sort by Final Net Worth
df_agg = df_agg.sort_values("Final Net Worth", ascending=False)

# --- STACKED BAR CHART ---
fig = go.Figure()

# Bar 1: Your Contributions (Hard Work)
fig.add_trace(go.Bar(
    name='Cash Contributed (Savings)',
    x=df_agg['Strategy'],
    y=df_agg['Total Contributed'],
    marker_color='rgb(55, 83, 109)' # Dark Blue
))

# Bar 2: Market Growth (Free Money)
fig.add_trace(go.Bar(
    name='Market Growth / Appreciation',
    x=df_agg['Strategy'],
    y=df_agg['Total Growth'],
    marker_color='rgb(26, 118, 255)' # Light Blue
))

fig.update_layout(
    title='<b>Sources of Wealth:</b> How much did you save vs. How much did it grow?',
    xaxis_title='Strategy',
    yaxis_title='Amount ($)',
    barmode='stack',
    template='plotly_white',
    height=500
)

# Add Annotation for Ratio
for i, row in df_agg.iterrows():
    ratio = row['Total Growth'] / row['Total Contributed']
    fig.add_annotation(
        x=row['Strategy'],
        y=row['Final Net Worth'],
        text=f"{ratio:.1f}x Multiplier",
        showarrow=False,
        yshift=10
    )

fig.show()

# --- TABLE VIEW ---
fig_tbl = go.Figure(data=[go.Table(
    header=dict(values=["Strategy", "Total Cash Invested", "Total Growth", "Final Net Worth"],
                fill_color='paleturquoise', align='left'),
    cells=dict(values=[
        df_agg['Strategy'],
        df_agg['Total Contributed'].apply(lambda x: f"${x:,.0f}"),
        df_agg['Total Growth'].apply(lambda x: f"${x:,.0f}"),
        df_agg['Final Net Worth'].apply(lambda x: f"${x:,.0f}")
    ], fill_color='lavender', align='left')
)])
fig_tbl.update_layout(title="<b>Portfolio Breakdown Summary</b>", height=300)
fig_tbl.show()

Running Portfolio Breakdown Analysis (100 runs)...
